# Problem set 6: Solving the Solow model

In [462]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# Tasks

## Solving matrix equations I

In [463]:
np.random.seed(1900)
n = 5
A = np.random.uniform(size=(n,n))
b = np.random.uniform(size=n)
c = np.random.uniform(size=n)
d = np.random.uniform(size=n)

**Question A:** Find the determinant of $[A \cdot A]^{-1}$

In [464]:
x = linalg.det(linalg.inv(A@A))
print(f'{x: .3f}')

 13132.549


**Answer:** see A1.py

**Question B:** Solve the following equation systems directly using **scipy**.

$$
\begin{aligned}
Ax &= b \\
Ax &= c \\
Ax &= d 
\end{aligned}
$$

In [465]:
variables = [('b', b), ('c', c), ('d', d)]

for i, j in variables:
    x = linalg.solve(A,j)
    print(f'x = {x} for A^(-1)*{i}')


x = [-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792] for A^(-1)*b
x = [ -7.72469812 -11.6354722   20.86796131   7.93343031   1.55909328] for A^(-1)*c
x = [-2.57374569 -4.00256301  7.93409587  2.56007481  1.25709881] for A^(-1)*d


**Answer:** A2.py

**Question C:** Solve the same equation systems as above using `linalg.lu_factor()` and `linalg.lu_solve()`. What is the benefit of this approach?

In [466]:
variables = [('b', b), ('c', c), ('d', d)]

LU, piv = linalg.lu_factor(A)

for i, j in variables:
    x = linalg.lu_solve((LU,piv),j)
    print(f'x = {x} for A^(-1)*{i}')

x = [-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792] for A^(-1)*b
x = [ -7.72469812 -11.6354722   20.86796131   7.93343031   1.55909328] for A^(-1)*c
x = [-2.57374569 -4.00256301  7.93409587  2.56007481  1.25709881] for A^(-1)*d


**Answer:** A3.py

## Solving matrix equations II

In [467]:
F = np.array([[2.0, 1.0, -1.0], [-3.0, -1.0, 2], [-2.0, 1.0, 2.0]])
e = np.array([8.0, -11.0, -3.0])

**Question:** Use the function `gauss_jordan()` in the `numecon_linalg` module located in this folder to solve

$$
Fx = e
$$

In [468]:
import numecon_linalg

x = np.column_stack((F,e))
numecon_linalg.gauss_jordan(x)
print('solution',x[:,-1])


solution [ 2.  3. -1.]


**Answer:** see A4.py

## Symbolic

**Question A:** Find

$$
\lim_{x \rightarrow 0} \frac{\sin(x)}{x}
$$

and

$$
\frac{\partial\sin(2x)}{\partial x} 
$$

In [469]:
# first equation
x = sm.symbols('x')

# write equation
sm.Limit((sm.sin(x)/(x)),x,0)

# evaluate function by writing lowercase limit
sm.limit((sm.sin(x)/(x)),x,0)



1

In [470]:
# second equation
x = sm.symbols('x')

# write equation
sm.Derivative(sm.sin(2*x),x)

# evaluate function by writing evaluate = True
sm.Derivative(sm.sin(2*x), x, evaluate = True)


2*cos(2*x)

**Answer:** A5.py

**Question B:** Solve the equation

$$ 
\frac{\sin(x)}{x} = 0
$$

In [471]:
x = sm.symbols('x')

# solve equation for x
sm.solve(sm.sin(x)/x, x)

[pi]

**Answer:** A6.py

# Problem: Solve the Solow model

## Introduction

Consider the **standard Solow-model** where:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$

## Steady state

Assume the production function is **Cobb-Douglas**.

**Question A:** Use **sympy** to find an analytical expression for the steady state, i.e. solve

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

In [472]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

In [473]:
# equation
ss = sm.Eq(k,(1/((1+n)*(1+g)))*(s*k**alpha+(1-delta)*k))

# solution
sol = sm.solve(ss, k)

# print solution
sol[0]


((delta + g*n + g + n)/s)**(1/(alpha - 1))

**Answer:** see A7.py

**Question B:** Turn you solution into a Python function called as `ss_func(s,g,n,delta,alpha)`. 

In [474]:
# create function
ss_func = sm.lambdify(args = (s, g, n, delta, alpha), expr = sol[0])

# create values which does not overwrite the original values
s_val = 0.2
g_val = 0.02
n_val = 0.01
alpha_val = 1/3
delta_val = 0.1

# test the function
ss_func_test = ss_func(s_val, g_val, n_val, delta_val, alpha_val)

print(f'There are {ss_func_test: .3f} units of capital in steady state')


There are  1.904 units of capital in steady state


**Answer:** A8.py

**Question C**: Find the steady state numerically using root-finding with `optimize.root_scalar`.

In [475]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

# solve for the root of k
objective = lambda k: k-(1/((1+n)*(1+g)))*(s*k**alpha+(1-delta)*k)

# find root: k in steady state
result = optimize.root_scalar(objective, bracket = [0.1,100], method = 'brentq')
print(result)


      converged: True
           flag: 'converged'
 function_calls: 11
     iterations: 10
           root: 1.9038315392313183


**Answer:** A9.py

**Question D:** Now assume the production function is CES. Find the steady state for $k$ for the various values of $\beta$ shown below.

In [476]:
betas = [-0.5,-0.25,-0.1,-0.05,0.05,0.1,0.25,0.5]

for beta in betas:
    objective = lambda k: k-(1/((1+n)*(1+g)))*(s*(alpha*k**beta+(1-alpha))**(1/beta)+(1-delta)*k)
    result = optimize.root_scalar(objective, bracket = [0.1,100], method = 'brentq')
    print(f'for beta = {beta:.3f} the steady state for k is',result.root)

for beta = -0.500 the steady state for k is 1.8471297000972997
for beta = -0.250 the steady state for k is 1.8733832627585918
for beta = -0.100 the steady state for k is 1.891085639765511
for beta = -0.050 the steady state for k is 1.8973581025712694
for beta = 0.050 the steady state for k is 1.9105159729244343
for beta = 0.100 the steady state for k is 1.9174221328177266
for beta = 0.250 the steady state for k is 1.9395902733677042
for beta = 0.500 the steady state for k is 1.9822334997701376


**Answer:** A10.py   